# 1. Collection des données

## 1.1 Installation des dépendances

In [ ]:
pip install labelme tensorflow opencv-python matplotlib albumentations

## 1.2 Collection des images

In [ ]:
import os
import time
import uuid
import cv2

In [ ]:
IMAGES_PATH = os.path.join('data','images')
RAW_IMAGES_PATH = os.path.join('data','raw_images')

NUMBER_OF_IMAGES = 5

In [ ]:
cap = cv2.VideoCapture(0)
time.sleep(5)

for i in range(NUMBER_OF_IMAGES):
    print(f"Capture de l'image {i}")
    time.sleep(2)
    
    success, frame = cap.read()
    if not success:
        print("Impossible de capturer l'image")
        break

    image_path = os.path.join(RAW_IMAGES_PATH, f'{uuid.uuid1()}.jpg')
    
    cv2.imwrite(image_path, frame)
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()

## 1.3 Annotation des images avec LabelMe

In [ ]:
!labelme